### **Definir el directorio de trabajo con Google Colab**
Define el directorio de trabajo como la carpeta `working_directory` de la carpeta compartida `DS4A-Team12` de Drive.

In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json
import re
import joblib
from utils import concatenar_codigos

### **Abrir base de datos de tomas nutricionales del ICBF (Dirección de Primera Infancia)**
Abre las primeras 98 columnas de todos los registros de la base de datos, preprocesados para asignarle a cada variable el tipo de dato correspondiente (archivo `TomasDPIpre.parquet` de 472.4 MB) como un dataframe de nombre `tom`.
Para preprocesar un archivo ir a [`Preprocesamiento_Tomas_.ipynb`](https://https://colab.research.google.com/drive/1ztyI5yfqs6LY7LOYXHr0q4WCrDdfotBs#scrollTo=KJM1uKZq71HA).

In [ ]:
time0 = time.time()
# Abre el diccionario con la estructura de datos definida para ambos datasets
with open('datatypes_dictionaries/dtypes_vars_toma.json', 'r') as file:
  dtypes_vars_toma = json.load(file)
with open('datatypes_dictionaries/dtypes_sociodemo.json', 'r') as file:
  dtypes_soc = json.load(file)
# Lee el archivo
tom = pd.read_parquet('clean_data/tomas.parquet')
tom = tom.astype(dtypes_vars_toma)
tom.info()

soc = pd.read_parquet('clean_data/sociodemo_filtered.parquet')
soc = soc.astype(dtypes_soc)
soc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19275081 entries, 0 to 19275080
Data columns (total 27 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   IdToma                          UInt32        
 1   Registro                        UInt32        
 2   Vigencia                        category      
 3   Toma                            category      
 4   Servicio                        category      
 5   FechaValoracionNutricional      datetime64[ns]
 6   EdadMeses                       float32       
 7   FechaMedicionPerimetroBraquial  datetime64[ns]
 8   MedicionPerimetroBraquial       float32       
 9   Peso                            float32       
 10  Talla                           float32       
 11  ZScoreTallaEdad                 float32       
 12  ZScorePesoEdad                  float32       
 13  ZScorePesoTalla                 float32       
 14  ZScoreIMC                       float32       
 

In [ ]:
tom['CodigoPesoTalla'] = tom['EstadoPesoTalla'].cat.codes
# Concatena los códigos de categoría de cada variable para cada beneficiario
his_zpt = tom.groupby('IdBeneficiario')['CodigoPesoTalla'].apply(concatenar_codigos)
# Agrega el 'IdBeneficiario' como la primera columna
his_zpt = his_zpt.reset_index(drop=False)
his_zpt.columns = ['IdBeneficiario', 'HistoriaPesoTalla']
his_zpt.to_parquet('historical_data/historia_peso_talla')
print(f'Historia del estado peso/talla codificada y guardada:' + \
      f'{time.time() - time0:.2f} s.')

Historia del estado peso/talla codificada y guardada:1706.11 s.


In [ ]:
# Desnutrición: Al menos un evento de desnutrición
# Definición de evento de desnutrición: criterio estricto (usual): z-score Peso/Talla <= -2
# Identificado por categorías 0 y 1 de EstadoPesoTalla
his_zpt['desnutricion'] = his_zpt['HistoriaPesoTalla'].\
  apply(lambda x: bool(re.match('[0-1]+', x)))

# Recuperación: Al menos un evento de no-desnutrición posterior a al menos un evento de desnutrición
his_zpt['recuperacion'] = his_zpt['HistoriaPesoTalla'].\
  apply(lambda x: bool(re.match('[0-1]+[2-6]+', x)))

# Reincidencia: Al menos un evento de desnutrición posterior a una recuperación
his_zpt['reincidencia'] = his_zpt['HistoriaPesoTalla'].\
  apply(lambda x: bool(re.match('[0-1]+[2-6]+[0-1]+', x)))

# Elimina la columna del histórico de fechas peso talla
his_zpt = his_zpt.drop(columns=['HistoriaPesoTalla'])

# Guarda el dataframe a un archivo parquet
his_zpt.to_parquet('clean_data/beneficiario_etiquetas')
print(f'Datos etiquetados y guardados:' + \
      f'{time.time() - time0:.2f} s.')

Datos etiquetados y guardados:2037.75 s.


In [ ]:
#Informalmente
os.chdir('/content/drive/MyDrive/DS4A-Team12/data')
tom_antiguo = pd.read_parquet('preprocessed_data/TomasDPI_tagged_data')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
idB_antiguos = tom_antiguo['IdBeneficiario']
idB_nuevos = set(his['IdBeneficiario']).difference(set(idB_antiguos))
idB_perdidos = set(idB_antiguos).difference(set(his['IdBeneficiario']))

In [ ]:
his_zpt[['desnutricion', 'recuperacion', 'reincidencia']].apply(sum, axis='index')

desnutricion    82020
recuperacion    71488
reincidencia     7178
dtype: int64

In [ ]:
tom['EstadoPesoTalla'].groupby(tom['Direccion']).value_counts(normalize=True, sort=False)

Direccion         EstadoPesoTalla             
Primera Infancia  Desnutrición aguda severa       0.003382
                  Desnutrición aguda moderada     0.009137
                  Riesgo de desnutrición aguda    0.055519
                  Peso adecuado para la talla     0.696579
                  Riesgo de sobrepeso             0.175772
                  Sobrepeso                       0.044555
                  Obesidad                        0.015056
Nutrición         Desnutrición aguda severa       0.011681
                  Desnutrición aguda moderada     0.037820
                  Riesgo de desnutrición aguda    0.209402
                  Peso adecuado para la talla     0.632325
                  Riesgo de sobrepeso             0.077806
                  Sobrepeso                       0.023288
                  Obesidad                        0.007679
Name: EstadoPesoTalla, dtype: float64

In [ ]:
# Cruza los datos socioeconómicos con las etiquetas
join = soc.merge(his_zpt, how='inner', on='IdBeneficiario')
join.info()

# Realiza conjuntos de entrenamiento y prueba para este conjunto
IdB_train = joblib.load('auxiliary_data/IdBeneficiario_train.pickle')
IdB_test = joblib.load('auxiliary_data/IdBeneficiario_test.pickle')

join_train = join[join['IdBeneficiario'].isin(IdB_train)]
join_test = join[join['IdBeneficiario'].isin(IdB_test)]
len(join_train), len(join_test)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1508999 entries, 0 to 1508998
Data columns (total 71 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Cod_clase                  1508999 non-null  category      
 1   Ind_grupo_sisben_4         1508999 non-null  category      
 2   Ind_nivel_sisben_4         1508999 non-null  category      
 3   Tip_vivienda               1508999 non-null  category      
 4   Ind_tiene_energia          1508999 non-null  category      
 5   Ind_tiene_alcantarillado   1508999 non-null  category      
 6   Ind_tiene_gas              1508999 non-null  category      
 7   Ind_tiene_recoleccion      1508999 non-null  category      
 8   Ind_tiene_acueducto        1508999 non-null  category      
 9   n_hogares_vivienda         1508999 non-null  UInt8         
 10  Tip_ocupa_vivienda         1508999 non-null  category      
 11  Ind_tiene_cocina           1508999 no

(1206993, 302006)

In [ ]:
join[['desnutricion', 'recuperacion', 'reincidencia']].apply(sum, axis='index')

desnutricion    36761
recuperacion    33139
reincidencia     3335
dtype: int64

In [ ]:
join.to_parquet('clean_data/sociodemo_etiquetas.parquet')
join_train.to_parquet('clean_data/sociodemo_etiquetas_train.parquet')
join_test.to_parquet('clean_data/sociodemo_etiquetas_test.parquet')